# Managing State in Multi-Agent Applications
In a multi-agent system, it's often useful to save the current state of agents, teams, and termination conditions. This allows us to load them back later, which is especially helpful in web applications where endpoints handle requests without storing persistent state.

In this guide, we'll explore how to save and load the state of agents and teams.

### Saving and Loading Agents
To save an agent’s state, we use the `save_state()` method. Let's create an assistant agent that shares fun facts about animals.

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an assistant agent
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a fun fact expert about animals.",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-mini",
    ),
)

# Ask for a fun fact about dolphins
response = await assistant_agent.on_messages(
    [TextMessage(content="Tell me a fun fact about dolphins.", source="user")], CancellationToken()
)
print(response.chat_message.content)


Dolphins are known for their exceptional intelligence and social behavior! One fun fact is that they have been observed using tools. For example, some bottlenose dolphins use marine sponges to protect their snouts while foraging on the ocean floor, helping them to search for prey in sandy environments without injuring themselves. This behavior showcases not only their problem-solving skills but also a level of cultural transmission, as young dolphins learn these techniques from their mothers!


In [2]:

# Save the agent's state
agent_state = await assistant_agent.save_state()
print(agent_state)

{'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Tell me a fun fact about dolphins.', 'source': 'user', 'type': 'UserMessage'}, {'content': 'Dolphins are known for their exceptional intelligence and social behavior! One fun fact is that they have been observed using tools. For example, some bottlenose dolphins use marine sponges to protect their snouts while foraging on the ocean floor, helping them to search for prey in sandy environments without injuring themselves. This behavior showcases not only their problem-solving skills but also a level of cultural transmission, as young dolphins learn these techniques from their mothers!', 'thought': None, 'source': 'assistant_agent', 'type': 'AssistantMessage'}]}}


In [3]:

# Create a new agent instance and load the saved state
new_assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a fun fact expert about animals.",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-mini",
    ),
)
await new_assistant_agent.load_state(agent_state)

# Ask about the previous fun fact
response = await new_assistant_agent.on_messages(
    [TextMessage(content="What was the fun fact you shared about dolphins?", source="user")], CancellationToken()
)
print(response.chat_message.content)


I shared that dolphins are known for their intelligence and social behavior, and one fun fact is that some bottlenose dolphins use marine sponges as tools to protect their snouts while foraging on the ocean floor. This behavior helps them search for prey in sandy areas without hurting themselves, demonstrating their problem-solving skills and cultural learning from mother to offspring!


### Saving and Loading Teams
We can also save and load the state of a team. When we save a team’s state, it includes the state of all agents within it.

##### Creating a Team
Let's create a team where an agent shares random animal facts.

In [ ]:
# Create an assistant agent
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a fun fact expert about animals.",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-mini",
    ),
)

# Define a team that cycles through agents
agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))

# Ask for an animal fact
stream = agent_team.run_stream(task="Tell me a surprising fact about octopuses.")
await Console(stream)

# Save the team's state
team_state = await agent_team.save_state()

---------- user ----------
Tell me a surprising fact about octopuses.
---------- assistant_agent ----------
A surprising fact about octopuses is that they have three hearts. Two of their hearts pump blood to the gills, where it picks up oxygen, and the third heart pumps the oxygenated blood to the rest of the body. Interestingly, when an octopus swims, the heart that delivers blood to the rest of the body actually stops beating, which is one reason why they prefer to crawl rather than swim—they tire less quickly that way!


### Resetting the Team
If we reset the team and ask about the last shared fact, it won’t remember because the state was not loaded.

In [5]:
# Reset the team
await agent_team.reset()

# Try asking again
stream = agent_team.run_stream(task="What was the last animal fact you shared?")
await Console(stream)

---------- user ----------
What was the last animal fact you shared?
---------- assistant_agent ----------
I don't have a memory of past interactions, but I can certainly share a fun animal fact with you now! Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body. This unique circulatory system helps supply their active bodies with the oxygen they need.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last animal fact you shared?', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=30, completion_tokens=70), metadata={}, content="I don't have a memory of past interactions, but I can certainly share a fun animal fact with you now! Did you know that octopuses have three hearts? Two pump blood to the gills, while the third pumps it to the rest of the body. This unique circulatory system helps supply their active bodies with the oxygen they need.", type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

### Loading the Team's State
Now, let's load the saved state and ask the same question again.

In [6]:
# Load the saved state
await agent_team.load_state(team_state)

# Ask again
stream = agent_team.run_stream(task="What was the last animal fact you shared?")
await Console(stream)


---------- user ----------
What was the last animal fact you shared?
---------- assistant_agent ----------
The last animal fact I shared was about octopuses, specifically highlighting that they have three hearts. Two hearts pump blood to the gills, and the third heart pumps it to the rest of the body. Additionally, when an octopus swims, the main heart stops beating, causing them to tire faster, which is why they prefer crawling over swimming.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last animal fact you shared?', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=141, completion_tokens=72), metadata={}, content='The last animal fact I shared was about octopuses, specifically highlighting that they have three hearts. Two hearts pump blood to the gills, and the third heart pumps it to the rest of the body. Additionally, when an octopus swims, the main heart stops beating, causing them to tire faster, which is why they prefer crawling over swimming.', type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

### Persisting State to a File
We can store the team’s state in a file (or a database) and reload it later.

In [7]:
import json

# Save state to a file
with open("team_state.json", "w") as f:
    json.dump(team_state, f)

# Load state from a file
with open("team_state.json", "r") as f:
    team_state = json.load(f)

# Create a new team and load the saved state
new_agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))
await new_agent_team.load_state(team_state)

# Ask again
stream = new_agent_team.run_stream(task="What was the last animal fact you shared?")
await Console(stream)


---------- user ----------
What was the last animal fact you shared?
---------- assistant_agent ----------
The last animal fact I shared was about octopuses, specifically that they have three hearts. Two hearts pump blood to the gills, and one pumps it to the rest of the body. Additionally, when an octopus swims, the main heart stops beating, which is why they prefer crawling to conserve energy.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the last animal fact you shared?', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=141, completion_tokens=64), metadata={}, content='The last animal fact I shared was about octopuses, specifically that they have three hearts. Two hearts pump blood to the gills, and one pumps it to the rest of the body. Additionally, when an octopus swims, the main heart stops beating, which is why they prefer crawling to conserve energy.', type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

## Summary
* Agents and teams can save their state to remember past interactions.
* Resetting a team loses previous context unless the state is reloaded.
* States can be saved to a file for long-term storage and later use.
* This approach is useful in applications where maintaining history across different sessions is important.